In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e12/sample_submission.csv
/kaggle/input/playground-series-s5e12/train.csv
/kaggle/input/playground-series-s5e12/test.csv


In [2]:
X_full = pd.read_csv("/kaggle/input/playground-series-s5e12/train.csv")

X_full.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [3]:
# Checking cardinality of columns
for col in X_full.columns:
    if(X_full[col].dtype == "object"):
        cardinality = X_full[col].nunique()
        print(str(col) + " " + str(cardinality))

gender 3
ethnicity 5
education_level 4
income_level 5
smoking_status 3
employment_status 4


In [4]:
from sklearn.model_selection import train_test_split

X_full = pd.read_csv("/kaggle/input/playground-series-s5e12/train.csv")
print(X_full.size)

y = X_full.diagnosed_diabetes
X_full.drop(["diagnosed_diabetes"], axis=1, inplace=True)

X_train_full, X_valid_full, y_train_full, y_valid_full = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 0)

categor_cols = [col for col in X_train_full.columns if X_train_full[col].dtype == "object"]

18200000


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.svm import LinearSVR, NuSVR
from sklearn.linear_model import SGDRegressor

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown = "ignore"))
])

# model = RandomForestRegressor(n_estimators = 100, random_state = 0) # head(10000) 0.3735
# model = DecisionTreeRegressor(random_state = 0)
# model = ExtraTreeRegressor()
# model = LinearSVR() # 0.375
# model = NuSVR() # 0.3755
model = SGDRegressor()

clf = Pipeline(steps = [
    ("preprocessor", categorical_transformer),
    ("model", model)
])

clf.fit(X_train_full, y_train_full)

prediction = clf.predict(X_valid_full)

In [6]:
from sklearn.metrics import mean_absolute_error

def err(y_valid, pred):
    p = np.where(pred >= 0.5, 1.0, 0.0)
    
    return mean_absolute_error(y_valid, p)

print("Error: ", err(y_valid_full, prediction))

Error:  0.3166


## Submitting results

In [7]:
X_test = pd.read_csv("/kaggle/input/playground-series-s5e12/test.csv")

In [8]:
preds_test = clf.predict(X_test)

print(preds_test)

output = pd.DataFrame({"id": X_test.id,
                      "diagnosed_diabetes": preds_test})
output.to_csv("submission.csv", index=False)

[0.45605223 0.69824877 0.73315652 ... 0.57966167 0.57384105 0.6181961 ]
